In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from src.models import ResNet, MNIST_CNN, CIFAR_CNN
from src.helpers import evaluate_rob_accuracy, evaluate_clean_accuracy, load_model, safe_model,_evaluate_model
from src.data_loader import load_torchvision_dataset, load_imagenette
#from src.pruning import identify_layers, _evaluate_sparsity

#device = torch.device("cuda:0")
device = torch.device("cpu")

dtype = torch.float32

# Initialization

In [24]:
#model = ResNet()
#model = MNIST_CNN()
model = CIFAR_CNN()

identifying layers


In [20]:
train_loader, test_loader = load_torchvision_dataset('CIFAR10')

Files already downloaded and verified
Files already downloaded and verified


In [21]:
PATH = './saved-models/CIFAR-baseline-150-epochs.pth'
model = load_model(model, PATH)

# Before

In [22]:
evaluate_clean_accuracy(model, test_loader, device)

(72.48, 0.0)

In [23]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

KeyboardInterrupt: 

# After

In [9]:
evaluate_clean_accuracy(model, test_loader, device)

(72.53, 0.0)

In [10]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

55.26

In [31]:
from torch.autograd import Variable
import time 


def fit_free(model, data_loader, epochs, device, number_of_replays=3, eps = 16/255):
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
    mean = torch.tensor(mean).view(3,1,1).expand(3,32,32)
    std = torch.tensor(std).view(3,1,1).expand(3,32,32)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters())
    
    pert_storage = torch.zeros((512,32,32,3))
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        t0 = time.time()
        running_loss, acc_epoch_loss, avg_epoch_loss, epoch_accuracy, acc_epoch_accuracy = 0.0, 0.0, 0.0, 0.0, 0.0
        #pert_storage = torch.zeros([512, 3, 32,32])
        for i, data in enumerate(data_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            print(inputs.shape[0])
            no_of_samples_in_batch = inputs.shape[0]
            

            # Mini Batch Replays
            for j in range(number_of_replays):
                noise_batch = Variable(pert_storage[:no_of_samples_in_batch], requires_grad=True)
                
                #print(inputs[0])
                
                adv_input = inputs+noise_batch[:no_of_samples_in_batch]
                adv_input.clamp_(0, 1.0)
                adv_input.sub_(mean).div_(std)
                
                #print(adv_input[0])

                # forward + backward + optimize
                outputs = model(adv_input)
                loss = criterion(outputs, labels)
                
                # zero the gradients
                optimizer.zero_grad()
                loss.backward(retain_graph=True)
                
                #craft adv pert
                pert = fgsm(noise_batch.grad)
                
                pert_storage += pert
                pert_storage.clamp_(-eps, eps)
                #update weights
                optimizer.step()

                    # print statistics
                running_loss += loss.item()
                accuracy = get_accuracy(labels, outputs)
                acc_epoch_loss += running_loss 
                avg_epoch_loss = acc_epoch_loss / (i+1)
                acc_epoch_accuracy += accuracy
                avg_epoch_accuracy = acc_epoch_accuracy / (i+1)
            if i%1 == 0:
                print('[%d, %5d] loss: %.5f, train_accuracy: %.2f' %(epoch + 1, i + 1, running_loss, accuracy))
            running_loss = 0.0
        t1 = time.time()
        accuracy, loss = _evaluate_model(model, test_data, device, criterion)
        print('duration:', t1-t0,'- train loss: ',avg_epoch_loss,' - train accuracy: ',avg_epoch_accuracy,' - validation accuracy: ', accuracy,' - validation loss: ', loss)
        print('duration: %d s - train loss: %.5f - train accuracy: %.2f - validation loss: %.2f - validation accuracy: %.2f ' %(t1-t0, avg_epoch_loss, avg_epoch_accuracy, loss, accuracy))
        
    print('Finished Training')
    return {
        'criterion': criterion,
        'optimizer': optimizer,
        'hist': 'Not implemented',
        'val_accuracy': accuracy
    }

def get_accuracy(labels, outputs):
    _, predicted = torch.max(outputs.data, 1)

    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

def fgsm(gradients, step_size=.05):
    return step_size*torch.sign(gradients)

In [32]:
fit_free(model, train_loader, 5, device, number_of_replays=3, eps = 16/255)

512


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 3

In [ ]:
global global_noise_data
global_noise_data = torch.zeros([512, 3, 32,32])
def fit_free( model,train_loader, criterion, optimizer, epoch, eps=8/255):
    global global_noise_data
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
    mean = torch.tensor(mean).view(3,1,1).expand(3,32,32)
    mean = torch.tensor(std).view(3,1,1).expand(3,32,32)
    # Initialize the meters
    # switch to train mode
    model.train()
    for i, (inputs, target) in enumerate(train_loader):
        running_loss, acc_epoch_loss, avg_epoch_loss, epoch_accuracy, acc_epoch_accuracy = 0.0, 0.0, 0.0, 0.0, 0.0
        inputs = inputs
        target = target
        for j in range(5):
            # Ascend on the global noise
            noise_batch = Variable(global_noise_data[0:inputs.size(0)], requires_grad=True)
            in1 = inputs + noise_batch
            in1.clamp_(0, 1.0)
            in1.sub_(mean).div_(std)
            output = model(in1)
            loss = criterion(output, target)
            
            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            
            # Update the noise for the next iteration
            pert = fgsm(noise_batch.grad, .1)
            global_noise_data[0:inputs.size(0)] += pert.data
            global_noise_data.clamp_(eps, eps)

            optimizer.step()
            # measure elapsed time
            accuracy = get_accuracy(target, output)
            acc_epoch_loss += running_loss 
            avg_epoch_loss = acc_epoch_loss / (i+1)
            acc_epoch_accuracy += accuracy
            avg_epoch_accuracy = acc_epoch_accuracy / (i+1)
            if i%2 == 0:
                print('[%d, %5d] loss: %.5f, train_accuracy: %.2f' %(epoch + 1, i + 1, running_loss, accuracy))
            running_loss = 0.0

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
    
    # Optimizer:
optimizer = torch.optim.Adam(model.parameters())
for epoch in range(10):
    fit_free(model, train_loader,criterion, optimizer, epoch)